<a href="https://colab.research.google.com/github/nekoradiation/National-Diet-Library/blob/master/KOKURITSU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>